In [0]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
from torch.autograd import Variable

In [0]:
users=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/users.dat',sep='::',engine='python',header=None,encoding='latin-1')

In [0]:
movies=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/movies.dat',sep='::',engine='python',header=None,encoding='latin-1')
ratings=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-1m/ratings.dat',sep='::',engine='python',header=None,encoding='latin-1')

In [0]:
training_set=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
test_set=pd.read_csv('/content/drive/My Drive/Machine Learning/Boltzmann_Machines/ml-100k/u1.test',delimiter='\t')


In [0]:
training_set=np.array(training_set,dtype=int)
test_set=np.array(test_set,dtype=int)

In [0]:
max_users=int(max(max(training_set[:,0]),max(test_set[:,0])))
max_movies=int(max(max(training_set[:,1]),max(test_set[:,1])))


In [0]:
def convert(data):
  convdata=np.zeros((max_users+1)*(max_movies+1)).reshape(max_users+1,max_movies+1)
  for i in range(0,len(data)):
    convdata[data[i][0]][data[i][1]]=data[i][2]
  list(convdata)  
  return convdata

In [44]:
tra=convert(training_set)
print(tra)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 ...
 [0. 5. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 5. ... 0. 0. 0.]]


In [0]:
training_set=convert(training_set)
test_set=convert(test_set)

In [0]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [0]:
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1

In [0]:
test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

In [0]:
class RBM():
  def __init__(self,nv,nh):
    self.W=torch.randn(nh,nv)
    self.A=torch.randn(1,nh)
    self.B=torch.randn(1,nv)
  def sample_h(self,x):
     wx = torch.mm(x, self.W.t())
     activation = wx + self.A.expand_as(wx)
     p_h_given_v = torch.sigmoid(activation)
     return p_h_given_v, torch.bernoulli(p_h_given_v) 
  def sample_v(self,y):
    wy = torch.mm( y, self.W )
    activation = wy + self.B.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h,torch.bernoulli(p_v_given_h) 
  def train(self, v0, vk, ph0, phk):
    self.W +=(torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.B += torch.sum((v0-vk), 0)
    self.A += torch.sum((ph0-phk), 0)

In [0]:
nv=len(training_set[0])
nh=200
batch_size=50
rbm=RBM(nv,nh)

In [88]:
nb_epochs=10
for i in range(1,nb_epochs+1):
  train_loss=0;
  s=0.
  for id_user in range(1,max_users-batch_size,batch_size):
    vk=training_set[id_user:id_user+batch_size]
    v0=training_set[id_user:id_user+batch_size]
    ph0,_=rbm.sample_h(v0)
    for k in range(10):
      _,hk=rbm.sample_h(vk)
      _,vk=rbm.sample_v(hk)
      vk[v0<0]=v0[v0<0]
    phk,_=rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)  
    train_loss+=torch.mean(torch.abs(v0[v0>=0]-vk[v0>=0]))
    s+=1.
  print("epoch: "+str(i)+" loss: "+str(train_loss/s))  

epoch: 1 loss: tensor(0.3234)
epoch: 2 loss: tensor(0.2482)
epoch: 3 loss: tensor(0.2508)
epoch: 4 loss: tensor(0.2480)
epoch: 5 loss: tensor(0.2500)
epoch: 6 loss: tensor(0.2503)
epoch: 7 loss: tensor(0.2473)
epoch: 8 loss: tensor(0.2460)
epoch: 9 loss: tensor(0.2470)
epoch: 10 loss: tensor(0.2457)


In [91]:
test_loss=0;
s=0.
for id_user in range(max_users):
    v=training_set[id_user:id_user+1]
    vt=test_set[id_user:id_user+1]
    if len(vt[vt>=0])>0:
      _,h=rbm.sample_h(v)
      _,v=rbm.sample_v(h)
      test_loss+=torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
      s+=1.
      
print("test loss: "+str(test_loss/s))  

test loss: tensor(0.2399)
